###  This notebook generates the 672 histogram pictures in one folder:
### "distribution of the element in hole" vs "distribution of the element in all dataset"
#### This "picture-set" helps to understand how the distribution correlates with geochemical anomalies



In [1]:
import sqlite3
import re
import matplotlib.pyplot as plt  
import os
import pandas as pd
import seaborn as sns
#Scott Halley's geochemistry data in geo3.db file

ls_db_file_name = 'geo3.db' # if geo3.db is in default notebook folder

#create database connection
connect_db = sqlite3.connect(ls_db_file_name)

# ! Sometime sqlite engine does not contain math functions like pow(x,y) etc. and you cannot use it in sql query. 
# It depends on the version of sqlite3.dll
# To calculate concentration, use python function pow(mant, expo)


#lest get a elements_list from table 'elements'
elements_list = []
ls_sql01 = "select name from elements"
cur = connect_db.cursor()
cur.execute(ls_sql01)
elements_list = cur.fetchall() # list of elements in elements_list

#lest get a holes_list from table 'holes'
holes_list = []
ls_sql02 = "select name from holes"
cur = connect_db.cursor()
cur.execute(ls_sql02)
holes_list = cur.fetchall() # list of elements in elements_list

In [2]:
#Function f_plot_hole_hist creates png histograms 
"""
arguments:
as_element_name - string value like "Ag"
as_labeled   - string value in set one of 2 variants "all", "labeled" - if labeled then only data with exclusion_ref=0
as_hole_name - string value is the name of the hole like "DDH005" 
as_sub_folder - subfolder in your python catalog with jpg scattergrams
connect_db - database connection object
f_plot_drill_scatter(ls_tmp_element_name,'E(A)',connect_db)
"""
def  f_plot_hole_hist(as_element_name,as_labeled,as_hole_name, as_sub_folder, connect_db):   
    #clear local variables
    ls_sql1 = ''
    ls_sql2 = ''
    ls_labeled = ''
    ls_tmp_color = ''
    ls_main = ''
    ls_chart_name = ls_main + as_element_name + ' in '+as_hole_name
    ls_x_label = ''  
    ls_y_label = as_element_name
    df1 = pd.DataFrame(None)
    df2 = pd.DataFrame(None)
    # adds "main" word to diagram title for main elements and mark x-axis with ppm or pct
    if ls_tmp_element_name in ['Al','Ca','Fe','K','Mg','Na','S','Ti','SiO2']:
        ls_main = 'main '
        ls_ppm='pct'
    else:
        ls_ppm='ppm'           
    # if labeled then we will add additional sql clause to query that retrive only pseudoclear data labeled in col exclusion_ref=0
    if as_labeled =='labeled':
        ls_labeled = " and t1.exclusion_ref=0 " 
    else: 
        ls_labeled = "" 

    # create the plot filename in ls_file_name2
    ls_file_name2 = as_element_name  + '-' +  as_hole_name        
#    ls_file_name2 = as_hole_name + '-' +  as_element_name

    # assemble sql query into ls_sql1 to extract data for specified element and _ONE_ bore hole   
    ls_sql1 = "select t1.mant,t2.from1,t2.to1  from data t1, samples t2, holes t3, elements t4 " + \
              "where t1.sample_ref=t2.id and t2.hole_ref=t3.id and t1.element_ref=t4.id and "+ \
              "t4.name='" + as_element_name + "' and " + \
              "t3.name='" + as_hole_name + "'" + ls_labeled  +"  order by t2.from1"  
    # assemble sql query into ls_sql2 to extract data for specified element and _ALL_ holes   
    ls_sql2 = "select t1.mant from data t1, elements t4 " + \
              "where t1.element_ref=t4.id and "+ \
              "t4.name='" + as_element_name+"'" + ls_labeled    

    # execute query ls_Sql1 to retrieve data to a dataframe df1
    query = connect_db.execute(ls_sql1)
    cols1 = [column[0] for column in query.description]
    df1 = pd.DataFrame.from_records(data = query.fetchall(), columns = cols1)
    
    # execute query ls_Sql2 to retrieve data to a dataframe df1
    query = connect_db.execute(ls_sql2)
    cols2 = [column[0] for column in query.description]
    df2 = pd.DataFrame.from_records(data = query.fetchall(), columns = cols2)
    
    # if no data then go out, exit
    ll_count1 = 0
    ll_count1 = df1['mant'].count()
    if ll_count1 == 0 :
        return 0
    ll_count2 = 0
    ll_count2 = df2['mant'].count()
    if ll_count2 == 0 :
        return 0
    
    sns.reset_defaults()
    # two histogramms: GRAY ALL holes , BLUE- only current Hole, in one plot
    # setting different parameters to adjust each grid
    # preparing 2 diagram in one figure with size and common x-axis hspace=0 knits two charts
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 13), gridspec_kw={'height_ratios': [1, 2],'hspace': 0})   
    # creating 2 statistical histograms in top of the plot
    ax0 = plt.subplot(2, 1, 1)
    sns_plot = sns.histplot(df2['mant'],stat = "density",bins = 30,color = "gray")  
    sns_plot = sns.histplot(df1['mant'],stat = "density",bins = 30,color = "blue")
    plt.legend(labels=["ALL holes",as_hole_name])
    sns_plot.set_title(ls_chart_name)
    # creating downhole plot of the concentration by hole depth   
    ax1 = plt.subplot(2, 1, 2, sharex = ax0)
    plt.plot(df1['mant'], -df1['from1'], c='blue')  
    # creating the axe name and the title with ls_chart_name, save to file    
    ax1.set_xlabel(ls_ppm) 
    ax1.set_ylabel('depth')
    # saving to file    
    fig = sns_plot.get_figure()
    fig.savefig(as_sub_folder + '/' + ls_file_name2 + '.png')
    # do not forget to clear plot objects close file to free up memory   
    fig.get_figure().clf()
    plt.close('all')

In [3]:
#specify labeled if only the data without statistical anomaly and without below LOD will be used, otherwise ls_labeled=""
ls_labeled='labeled'
#specify subfolder where you want to create pictures
ls_sub_folder='pic3' #pictures with diagrams will be uploaded to this subfolder
#if folder does not exist it will be created with mkdir
if not(os.path.isdir(ls_sub_folder)) :
    os.mkdir(ls_sub_folder)    
# i this used to clear values ls_tmp_element_name from commas, parentheses using regexp 
reg = re.compile('[^a-zA-Z0-9]')    # sorry, then I'll see how to work correctly with a dictionary and a list without such dirty tricks
# two nested loops on elements and wells 
for i in range(0, len(elements_list)):    
    for j in range(0, len(holes_list)):  
#for i in range(0, 1):    
#    for j in range(0, 4):          
            ls_tmp_element_name = ''
            ls_tmp_hole_name = '' 
            #this regexps clear elements_list from extra characters        
            ls_tmp_element_name = reg.sub('', str(elements_list[i]))   
            ls_tmp_hole_name = reg.sub('', str(holes_list[j]))           
            #create diagram files to each element 
            f_plot_hole_hist(ls_tmp_element_name,ls_labeled,ls_tmp_hole_name,ls_sub_folder, connect_db)
